In [2]:
!pip install flask econml scikit-learn matplotlib


In [1]:
from flask import Flask, request, render_template_string
import pandas as pd
import numpy as np
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
import io
import base64
import threading

app = Flask(__name__)


In [2]:
!pip install causalml 

In [3]:
def perform_econml_dml(df):
    try:
        #encoding gender column
        df['gender'] = df['gender'].map({'male': 1, 'female': 0})
        X = df[['age', 'gender']]    
        treatment = df['treatment']   
        y = df['outcome']  
        est = LinearDML(model_y=LassoCV(),
                model_t=RandomForestRegressor(),
                discrete_treatment=False,
                random_state=0)
        est.fit(X=X, T=treatment, Y=y)
        ate_effects = est.effect(X)  
        ate_value = np.mean(ate_effects)
         # Plot
        fig, ax = plt.subplots(figsize=(6, 3))
        ax.hlines(y=ate_value, xmin=0, xmax=1, colors="green", linestyles="--", label=f"ATE = {ate_value:.2f}")
        ax.set_title("Average Treatment Effect (ATE)")
        ax.set_xlabel("Samples")
        ax.set_ylabel("Treatment Effect")
        ax.set_ylim(ate_value - 0.5, ate_value + 0.5)
        ax.legend()

        buf = io.BytesIO()
        plt.tight_layout()
        plt.savefig(buf, format="png")
        buf.seek(0)
        image_base64 = base64.b64encode(buf.read()).decode('utf-8')
        buf.close()
        plt.close(fig)

        return ate_value, image_base64
    except Exception as e:
        return f"Error: {str(e)}", None

In [4]:


upload_form = '''
<!doctype html>
<html lang="en">
<head>
    <title>EconML DML CSV Upload</title>
    <style>
        :root {
            --primary: #4361ee;
            --secondary: #3f37c9;
            --accent: #4cc9f0;
            --light: #f8f9fa;
            --dark: #212529;
            --gray: #6c757d;
            --danger: #f72585;
            --success: #4ade80;
            --border-radius: 8px;
            --box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            --transition: all 0.3s ease;
        }
        
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        
        body {
            background-color: #f5f7fa;
            color: var(--dark);
            line-height: 1.6;
            padding: 20px;
            max-width: 900px;
            margin: 0 auto;
        }
        
        .container {
            background-color: white;
            border-radius: var(--border-radius);
            box-shadow: var(--box-shadow);
            padding: 2.5rem;
            margin-top: 2rem;
        }
        
        h2 {
            color: var(--primary);
            margin-bottom: 1.5rem;
            text-align: center;
            font-weight: 600;
            font-size: 1.8rem;
        }
        
        .upload-form {
            display: flex;
            flex-direction: column;
            gap: 1.5rem;
            margin-bottom: 2rem;
        }
        
        .file-input-wrapper {
            position: relative;
            overflow: hidden;
            display: flex;
            flex-direction: column;
            gap: 0.5rem;
        }
        
        .file-input-label {
            font-weight: 500;
            color: var(--gray);
        }
        
        .file-input {
            width: 100%;
            padding: 0.75rem;
            border: 1px solid #ced4da;
            border-radius: var(--border-radius);
            background-color: var(--light);
            transition: var(--transition);
        }
        
        .file-input:focus {
            outline: none;
            border-color: var(--primary);
            box-shadow: 0 0 0 0.2rem rgba(67, 97, 238, 0.25);
        }
        
        .submit-btn {
            background-color: var(--primary);
            color: white;
            border: none;
            padding: 0.75rem 1.5rem;
            border-radius: var(--border-radius);
            cursor: pointer;
            font-size: 1rem;
            font-weight: 500;
            transition: var(--transition);
            width: 100%;
        }
        
        .submit-btn:hover {
            background-color: var(--secondary);
            transform: translateY(-2px);
        }
        
        .divider {
            height: 1px;
            background-color: #e9ecef;
            margin: 1.5rem 0;
            border: none;
        }
        
        .results {
            margin-top: 1.5rem;
            animation: fadeIn 0.5s ease;
        }
        
        .results h3 {
            color: var(--secondary);
            margin-bottom: 1rem;
            font-size: 1.4rem;
        }
        
        .results-list {
            list-style: none;
            margin-bottom: 1.5rem;
        }
        
        .results-list li {
            padding: 0.75rem 1rem;
            background-color: var(--light);
            border-radius: var(--border-radius);
            margin-bottom: 0.75rem;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }
        
        .results-list strong {
            color: var(--primary);
        }
        
        .plot-container {
            margin: 1.5rem 0;
            text-align: center;
        }
        
        .plot-container img {
            max-width: 100%;
            border-radius: var(--border-radius);
            box-shadow: var(--box-shadow);
            border: 1px solid #eee;
        }
        
        .error-message {
            color: var(--danger);
            background-color: rgba(247, 37, 133, 0.1);
            padding: 1rem;
            border-radius: var(--border-radius);
            margin-top: 1.5rem;
            border-left: 4px solid var(--danger);
            font-weight: 500;
        }
        
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }
        
        .method-tag {
            display: inline-block;
            padding: 0.25rem 0.5rem;
            border-radius: 4px;
            font-size: 0.75rem;
            font-weight: 600;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }
        
        .dml-tag {
            background-color: rgba(67, 97, 238, 0.1);
            color: var(--primary);
        }
        
        .psm-tag {
            background-color: rgba(76, 201, 240, 0.1);
            color: var(--accent);
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Upload CSV for Double Machine Learning (EconML)</h2>
        
        <form method="post" enctype="multipart/form-data" class="upload-form">
            <div class="file-input-wrapper">
                <label for="csv_file" class="file-input-label">Select CSV file:</label>
                <input type="file" name="csv_file" id="csv_file" class="file-input" accept=".csv" required>
            </div>
            <button type="submit" class="submit-btn">Analyze Data</button>
        </form>
        
        {% if ate or ate_psm %}
            <hr class="divider">
            
            <div class="results">
                <h3>Estimated Treatment Effects</h3>
                
                <ul class="results-list">
                    {% if ate %}
                    <li>
                        <span><span class="method-tag dml-tag">DML</span> Average Treatment Effect</span>
                        <strong>{{ ate }}</strong>
                    </li>
                    {% endif %}
                    
                    {% if ate_psm %}
                    <li>
                        <span><span class="method-tag psm-tag">PSM</span> Average Treatment Effect</span>
                        <strong>{{ ate_psm }}</strong>
                    </li>
                    {% endif %}
                </ul>
                
                {% if plot_data %}
                <div class="plot-container">
                    <img src="data:image/png;base64,{{ plot_data }}" alt="ATE Visualization">
                </div>
                {% endif %}
            </div>
        {% endif %}
        
        {% if error %}
            <div class="error-message">
                {{ error }}
            </div>
        {% endif %}
    </div>
</body>
</html>
'''


@app.route('/', methods=['GET', 'POST'])
def upload_and_analyze():
    ate = None
   
    plot_data = None
    error = None

    if request.method == 'POST':
        file = request.files.get('csv_file')
        if file and file.filename.endswith('.csv'):
            try:
                df = pd.read_csv(file)
                
                # Call both backend functions
                ate_result, plot_data = perform_econml_dml(df)
            

                # Error handling
                if isinstance(ate_result, str):
                    error = f"DML Error: {ate_result}"
               
                else:
                    ate = f"{ate_result:.4f}"
                    
            except Exception as e:
                error = f"Processing error: {str(e)}"
        else:
            error = "Please upload a valid CSV file."

    return render_template_string(upload_form, ate=ate, plot_data=plot_data, error=error)


In [5]:
def run_flask():
    app.run(port=5000, debug=True,use_reloader=False)

threading.Thread(target=run_flask).start()
